# pYPKa_Z_RPL14B and pYPKa_E_RPL14B

This notebook describe the construction of E. coli vectors [pYPKa_Z_RPL14B](pYPKa_Z_RPL14B.gb) and [pYPKa_E_RPL14B](pYPKa_E_RPL14B.gb)

The insert defined below is cloned in pYPKa using the blunt restriction
enzymes [ZraI](http://rebase.neb.com/rebase/enz/ZraI.html) and [EcoRV](http://rebase.neb.com/rebase/enz/EcoRV.html) in
two different plasmids. The insert cloned in ZraI will be used as a promoter, while in the EcoRV site it will be used as a
terminator.

![pYPKa_Z and pYPKa_E plasmids](pYPK_ZE.png "pYPKa_Z and pYPKa_E plasmids")

The [pydna](https://pypi.python.org/pypi/pydna/) package is imported in the code cell below.
There is a [publication](http://www.biomedcentral.com/1471-2105/16/142) describing pydna as well as
[documentation](http://pydna.readthedocs.io/) available online.
Pydna is developed on [Github](https://github.com/BjornFJohansson/pydna).

In [1]:
from pydna.readers import read
from pydna.parsers import parse_primers
from pydna.genbank import Genbank
from pydna.amplify import pcr
from pydna.amplify import Anneal

The vector backbone pYPKa is read from a local [file](pYPKa.gb).

In [2]:
pYPKa = read("pYPKa.gb")

Both restriction enzymes are imported from [Biopython](http://biopython.org/wiki/Main_Page)

In [3]:
from Bio.Restriction import ZraI, EcoRV

The vector is cut with both enzymes.

In [4]:
pYPKa_ZraI  = pYPKa.linearize(ZraI)
pYPKa_EcoRV = pYPKa.linearize(EcoRV)

The template below comes from a Genbank [record](http://www.ncbi.nlm.nih.gov/nuccore/BK006934.2).
Access to Genbank is needed in order to download the template.
If you execute this script, change the email address below to your own.
Always tell Genbank who you are, when using their web service.

In [5]:
gb = Genbank("bjornjobb@gmail.com")

The template is downloaded from Genbank below.

In [6]:
template = gb.nucleotide("BK006934.2 REGION: 103971..104276")

The template is a 306 bp linear DNA fragment.

In [7]:
template

Gbank(BK006934.2  103971-104276)(-306)

The insert has the sequence shown below.

In [8]:
print(str(template.seq))

CCCTTCAACTTATATAGCACCTTTACATATCTTTAATTTTTACGAAGAGCATTTTATTCAACTTTTGTACCTTTAAAGACGGTTTTCAGCCCACACATTTTTTGTTATTTTACACCACACAGTAAAACTTTCTTGCTTTTTTTTCTTGAGACTCCCCGACCATTGTTCGATATTCCAATTTTCGCAATCTTCAGAAAAGGGTATCCTAACATTCTAAATTAATAATAAAGAAGAGAGGCTACAAGCATTTATTAGTGTGAAGCATTTAAAGTAACTTCAGGAAATAAACGCGCAAATAAACCAAAA


The seguid checksum of the template should be

```LR0DmD7j9e5Pw73rnVuhk4XsqwY```

In [9]:
template.seguid()

LR0DmD7j9e5Pw73rnVuhk4XsqwY

Two primers are used to amplify the insert:

In [10]:
fp,rp = parse_primers(""">761_RPL14B_fw
                         ttaaatCCCTTCAACTTATATAGCACCT
                         >762_RPL14B_rv
                         taattaaTTTTGGTTTATTTGCGC""")

PCR to create the insert using the primers above.

In [11]:
prd = pcr(fp, rp, template)
assert str(fp.seq) in prd

The PCR product has this length in bp.

In [12]:
len(prd)

319

A figure of the primers annealing on template.

In [13]:
prd.figure()

      5CCCTTCAACTTATATAGCACCT...GCGCAAATAAACCAAAA3
                                ||||||||||||||||| tm 50.4 (dbd) 59.0
                               3CGCGTTTATTTGGTTTTaattaat5
5ttaaatCCCTTCAACTTATATAGCACCT3
       |||||||||||||||||||||| tm 54.7 (dbd) 59.4
      3GGGAAGTTGAATATATCGTGGA...CGCGTTTATTTGGTTTT5

A suggested PCR program.

In [14]:
prd.program()


Taq (rate 30 nt/s) 35 cycles             |319bp
95.0°C    |95.0°C                 |      |Tm formula: Biopython Tm_NN
|_________|_____          72.0°C  |72.0°C|SaltC 50mM
| 03min00s|30s  \         ________|______|Primer1C 1.0µM
|         |      \ 49.5°C/ 0min10s| 5min |Primer2C 1.0µM
|         |       \_____/         |      |GC 30%
|         |         30s           |      |4-12°C

The final vectors are:

In [15]:
pYPKa_Z_RPL14B = (pYPKa_ZraI  + prd).looped().synced(pYPKa)
pYPKa_E_RPL14B = (pYPKa_EcoRV + prd).looped().synced(pYPKa)

The final vectors with reverse inserts are created below. These vectors theoretically make up
fifty percent of the clones. The PCR strategy below is used to identify clones with the insert
in the correct orientation.

In [16]:
pYPKa_Z_RPL14Bb = (pYPKa_ZraI  + prd.rc()).looped().synced(pYPKa)
pYPKa_E_RPL14Bb = (pYPKa_EcoRV + prd.rc()).looped().synced(pYPKa)

A combination of standard primers and the primers above are
used for the strategy to identify correct clones.
Standard primers are listed [here](primers.fasta).

In [17]:
p = { x.id: x for x in parse_primers(""">577
                                         gttctgatcctcgagcatcttaagaattc
                                         >578
                                         gttcttgtctcattgccacattcataagt
                                         >468
                                         gtcgaggaacgccaggttgcccact
                                         >467
                                         ATTTAAatcctgatgcgtttgtctgcacaga
                                         >567
                                         GTcggctgcaggtcactagtgag
                                         >568
                                         GTGCcatctgtgcagacaaacg
                                         >775
                                         gcggccgctgacTTAAAT
                                         >778
                                         ggtaaatccggatTAATTAA
                                         >342
                                         CCTTTTTACGGTTCCTGGCCT""") }

## Diagnostic PCR confirmation

The correct structure of pYPKa_Z_RPL14B is confirmed by PCR using standard primers
577 and 342 that are vector specific together with the RPL14Bfw primer specific for the insert
in a multiplex PCR reaction with
all three primers present.

Two PCR products are expected if the insert was cloned, the sizes depend
on the orientation. If the vector is empty or contains another insert, only one
product is formed.

#### Expected PCR products sizes from pYPKa_Z_RPL14B:

pYPKa_Z_RPL14B with insert in correct orientation.

In [18]:
Anneal( (p['577'], p['342'], fp), pYPKa_Z_RPL14B).products

[Amplicon(1253), Amplicon(1085)]

pYPKa_Z_RPL14B with insert in reverse orientation.

In [19]:
Anneal( (p['577'], p['342'], fp), pYPKa_Z_RPL14Bb).products

[Amplicon(1253), Amplicon(487)]

Empty pYPKa clone.

In [20]:
Anneal( (p['577'], p['342'], fp), pYPKa).products

[Amplicon(934)]

#### Expected PCR products sizes pYPKa_E_RPL14B:

pYPKa_E_RPL14B with insert in correct orientation.

In [21]:
Anneal( (p['577'], p['342'], fp), pYPKa_E_RPL14B).products

[Amplicon(1253), Amplicon(1004)]

pYPKa_E_RPL14B with insert in reverse orientation.

In [22]:
Anneal( (p['577'], p['342'], fp), pYPKa_E_RPL14Bb).products


[Amplicon(1253), Amplicon(568)]

Calculate cseguid checksums for the resulting plasmids for future reference.
Cseguid is a checksum that uniquely describes a circular double stranded
sequence.

pYPKa_Z_RPL14B.cseguid() should be F0xUoQOyoSAy73P3wt3ChLBrQdE

In [23]:
print(pYPKa_Z_RPL14B.cseguid())
assert pYPKa_Z_RPL14B.cseguid() == "F0xUoQOyoSAy73P3wt3ChLBrQdE"

F0xUoQOyoSAy73P3wt3ChLBrQdE


pYPKa_E_RPL14B.cseguid() should be wGdYxQYXitrDNTt33F3U6TyIzEw

In [24]:
print(pYPKa_E_RPL14B.cseguid())
assert pYPKa_E_RPL14B.cseguid() == "wGdYxQYXitrDNTt33F3U6TyIzEw"

wGdYxQYXitrDNTt33F3U6TyIzEw


The sequences are named based on the name of the cloned insert.

In [25]:
pYPKa_Z_RPL14B.locus = "pYPKa_Z_RPL14B"[:16]
pYPKa_E_RPL14B.locus = "pYPKa_Z_RPL14B"[:16]

Stamp sequence with cseguid checksum. This can be used to verify the
integrity of the sequence file.

In [26]:
pYPKa_Z_RPL14B.stamp()
pYPKa_E_RPL14B.stamp()

cSEGUID_wGdYxQYXitrDNTt33F3U6TyIzEw

Sequences are written to local files.

In [27]:
pYPKa_Z_RPL14B.write("pYPKa_Z_RPL14B.gb")
pYPKa_E_RPL14B.write("pYPKa_E_RPL14B.gb")

pYPKa_Z_RPL14B.gb

pYPKa_E_RPL14B.gb

# [pYPKa_Z_RPL14B](pYPKa_Z_RPL14B.gb)

In [28]:
from pydna.readers import read
reloaded = read("pYPKa_Z_RPL14B.gb")
reloaded.verify_stamp()

cSEGUID_F0xUoQOyoSAy73P3wt3ChLBrQdE

# [pYPKa_E_RPL14B](pYPKa_E_RPL14B.gb)

In [29]:
from pydna.readers import read
reloaded = read("pYPKa_E_RPL14B.gb")
reloaded.verify_stamp()

cSEGUID_wGdYxQYXitrDNTt33F3U6TyIzEw